In [1]:
import subprocess
import pandas as pd
import re

In [2]:
import subprocess
import pandas as pd

def classify_inadmissibility(df, text_column="unofficial_text", model="llama3"):
    """
    Classify each court case using LLaMA 3 via subprocess by:
    1. Extracting numbered sections starting from [1].
    2. Summarizing the extracted text.
    3. Feeding the summary into a classification prompt.

    This version returns the raw classification output from the model.

    Parameters
    ----------
    df : pandas.DataFrame
        The DataFrame containing court case texts.
    text_column : str, optional
        The name of the column containing the court text (default is "unofficial_text").
    model : str, optional
        The name of the language model to use with Ollama (default is "llama3").

    Returns
    -------
    pandas.DataFrame
        A DataFrame with an added column 'inadmissibility' containing the raw model output.
    """

    def extract_numbered_lines(text):
        lines = text.splitlines()
        extracted = []
        start_extracting = False

        for line in lines:
            line_strip = line.strip()
            if not start_extracting:
                # Start if line contains "[1]" anywhere
                if "[1]" in line_strip:
                    extracted.append(line)
                    start_extracting = True
            else:
                # Continue only if line starts with [number] or number
                if line_strip.startswith("[") and line_strip[1:line_strip.find("]")].isdigit():
                    extracted.append(line)
                elif line_strip and line_strip[0].isdigit():
                    extracted.append(line)
                else:
                    break
        return "\n".join(extracted)

    def run_ollama(prompt):
        try:
            result = subprocess.run(
                ["ollama", "run", model],
                input=prompt.encode(),
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE
            )
            return result.stdout.decode().strip()
        except Exception as e:
            print(f"Subprocess error: {e}")
            return "subprocess_error"

    def generate_summary(text):
        prompt = f"""
You are a legal analyst specializing in Canadian immigration law.

Summarize the following court case in one sentence, clearly stating what the case is about.

Case Text:
{text}

Summary:
"""
        return run_ollama(prompt)

    def classify_summary(summary):
        prompt = f"""
You are a Canadian immigration law expert.

Based on the following summary of a legal case, classify whether the case involves 
a judicial review of an inadmissibility decision under Canadian immigration law.


Respond only with one of the following:
Inadmissibility
Not Inadmissibility

Summary:
{summary}

Classification:
"""
        return run_ollama(prompt)

    df = df.copy()
    df["inadmissibility"] = None

    for idx, row in df.iterrows():
        full_text = row[text_column]
        limited_text = extract_numbered_lines(full_text)
        summary = generate_summary(limited_text)
        raw_output = classify_summary(summary)
        df.at[idx, "inadmissibility"] = raw_output
        print(f"Row {idx} classified.")

    return df


In [3]:
df = pd.read_excel("/Users/husain/Downloads/court_cases.xlsx")
df

,Unnamed: 0,citation,dataset,year,language,document_date,source_url,judges,city_heard,outcome,inadmissibility_ground,unofficial_text
0,0,2014 FC 105,FC,2014,en,2014-01-30,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,['Strickland'],Ottawa,dismissed,['misrepresentation'],Muthui v. Canada (Citizenship and Immigration)...
1,1,2014 FC 112,FC,2014,en,2014-02-04,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,['Simon Noël'],Ottawa,dismissed,['other'],Adewole v. Canada (Citizenship and Immigration...
2,2,2014 FC 1137,FC,2014,en,2014-11-25,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,['Diner'],Ottawa,dismissed,['human_rights'],Sidhu v. Canada (Citizenship and Immigration)\...
3,3,2014 FC 1146,FC,2014,en,2014-11-28,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,['Shore'],Ottawa,dismissed,['security'],Abebe v. Canada (Citizenship and Immigration)\...
4,4,2014 FC 1147,FC,2014,en,2014-11-28,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,['Barnes'],Vancouver,dismissed,['misrepresentation'],Bundhel v. Canada (Citizenship and Immigration...
...,...,...,...,...,...,...,...,...,...,...,...,...
1133,1133,2024 FC 967,FC,2024,en,2024-06-21,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,['Ngo'],Ottawa,dismissed,['security'],Hamid v. Canada (Citizenship and Immigration)\...
1134,1134,2024 FC 971,FC,2024,en,2024-06-21,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,['Sadrehashemi'],Ottawa,dismissed,['security'],Singh v. Canada (Citizenship and Immigration)\...
1135,1135,2024 FC 980,FC,2024,en,2024-06-24,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,['McDonald'],Ottawa,dismissed,['non_compliance'],Akinokun v. Canada (Citizenship and Immigratio...
1136,1136,2024 FC 998,FC,2024,en,2024-06-26,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,['Grant'],Toronto,dismissed,['misrepresentation'],Singh Rai v. Canada (Citizenship and Immigrati...


In [4]:
inadmissibility_df = classify_inadmissibility(df)

Row 0 classified.
Row 1 classified.
Row 2 classified.
Row 3 classified.
Row 4 classified.
Row 5 classified.
Row 6 classified.
Row 7 classified.
Row 8 classified.
Row 9 classified.
Row 10 classified.
Row 11 classified.
Row 12 classified.
Row 13 classified.
Row 14 classified.
Row 15 classified.
Row 16 classified.
Row 17 classified.
Row 18 classified.
Row 19 classified.
Row 20 classified.
Row 21 classified.
Row 22 classified.
Row 23 classified.
Row 24 classified.
Row 25 classified.
Row 26 classified.
Row 27 classified.
Row 28 classified.
Row 29 classified.
Row 30 classified.
Row 31 classified.
Row 32 classified.
Row 33 classified.
Row 34 classified.
Row 35 classified.
Row 36 classified.
Row 37 classified.
Row 38 classified.
Row 39 classified.
Row 40 classified.
Row 41 classified.
Row 42 classified.
Row 43 classified.
Row 44 classified.
Row 45 classified.
Row 46 classified.
Row 47 classified.
Row 48 classified.
Row 49 classified.
Row 50 classified.
Row 51 classified.
Row 52 classified.
Row

In [5]:
inadmissibility_df

,Unnamed: 0,citation,dataset,year,language,document_date,source_url,judges,city_heard,outcome,inadmissibility_ground,unofficial_text,inadmissibility
0,0,2014 FC 105,FC,2014,en,2014-01-30,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,['Strickland'],Ottawa,dismissed,['misrepresentation'],Muthui v. Canada (Citizenship and Immigration)...,Inadmissibility
1,1,2014 FC 112,FC,2014,en,2014-02-04,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,['Simon Noël'],Ottawa,dismissed,['other'],Adewole v. Canada (Citizenship and Immigration...,Not Inadmissibility
2,2,2014 FC 1137,FC,2014,en,2014-11-25,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,['Diner'],Ottawa,dismissed,['human_rights'],Sidhu v. Canada (Citizenship and Immigration)\...,Not Inadmissibility
3,3,2014 FC 1146,FC,2014,en,2014-11-28,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,['Shore'],Ottawa,dismissed,['security'],Abebe v. Canada (Citizenship and Immigration)\...,Inadmissibility
4,4,2014 FC 1147,FC,2014,en,2014-11-28,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,['Barnes'],Vancouver,dismissed,['misrepresentation'],Bundhel v. Canada (Citizenship and Immigration...,Inadmissibility
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1133,1133,2024 FC 967,FC,2024,en,2024-06-21,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,['Ngo'],Ottawa,dismissed,['security'],Hamid v. Canada (Citizenship and Immigration)\...,Inadmissibility
1134,1134,2024 FC 971,FC,2024,en,2024-06-21,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,['Sadrehashemi'],Ottawa,dismissed,['security'],Singh v. Canada (Citizenship and Immigration)\...,Inadmissibility
1135,1135,2024 FC 980,FC,2024,en,2024-06-24,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,['McDonald'],Ottawa,dismissed,['non_compliance'],Akinokun v. Canada (Citizenship and Immigratio...,Inadmissibility
1136,1136,2024 FC 998,FC,2024,en,2024-06-26,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,['Grant'],Toronto,dismissed,['misrepresentation'],Singh Rai v. Canada (Citizenship and Immigrati...,Inadmissibility
